In [1]:
import cv2
import numpy as np
from imutils import perspective
from imutils import contours
from scipy.spatial import distance as dist
def midpoint(ptA, ptB):
   return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
refObj=None
colors = (255, 255, 0)
image = cv2.imread('abc.jpg')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(grayImage)
if len(faces) != 0:
    print("Number of faces detected: " + str(faces.shape[0]))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        box = np.array([x, y, x+w,y,x + w,y + h,x,y+h],dtype="int").reshape(4,2)
        box = perspective.order_points(box)
        cX = np.average(box[:, 0])
        cY = np.average(box[:, 1])
        if refObj is None:
            (tl, tr, br, bl) = box
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)
            D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
            refObj = (box, (cX, cY), D / 8)
            continue
    orig = image.copy()
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
    cv2.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 2)
    refCoords = np.vstack([refObj[0], refObj[1]])
    objCoords = np.vstack([box, (cX, cY)])
    for ((xA, yA), (xB, yB),i) in zip(refCoords, objCoords,range(0,5)):
        if(i==4):
            cv2.circle(orig, (int(xA), int(yA)), 5, colors, -1)
            cv2.circle(orig, (int(xB), int(yB)), 5, colors, -1)
            cv2.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),colors, 2)
            D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]
            (mX, mY) = midpoint((xA, yA), (xB, yB))
            cv2.putText(orig, "{:.1f}feet".format(D//12), (int(mX), int(mY - 10)),cv2.FONT_HERSHEY_SIMPLEX, 0.55, colors, 2)
            cv2.imshow("Image", orig)
            cv2.imwrite('social_distance.jpg',orig)
            cv2.waitKey(0)


Number of faces detected: 8


NameError: name 'cap' is not defined

In [1]:
import cv2
import math

cap = cv2.VideoCapture(0)

face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

print("******************************************************")
print("         Welcome to the Social Distancing App         ")
print("******************************************************")
print("\n")

print("For this model initial calibration was done by taking width of object to be 17cm and the distance of 30cm away from camera so as to calculate the focal length. And then this focal length was used for calculating distance between the object and camera.")

while True:    
    centroid = []
    person_in_contact=""
    status,photo = cap.read()
    face_cor = face_model.detectMultiScale(photo)
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    text="Total Face Detected " +str(len(face_cor))
    centroid.clear()
    i = 1
    if len(face_cor) == 0:
        pass
    else:
        for (x,y,w,h) in face_cor:    
            x1 = x
            y1 = y
            x2 = x + w
            y2 = y + h
            centroid.append((int((x2+x1)/2), int((y2+y1)/2)))
            #distance was calculated using focal length. focal length = 400cm. Focal_length = (Pixcel_width x  Distance from camera) / Width of obejct . width =17cm distance=30cm, pixcel=200px
            dis = "dis= " + "{:.2f}".format(17*400/(x2-x1)) + " cm"
            cv2.rectangle(photo, (x1,y1), (x2,y2), [0,255,0], 3)
            cv2.putText(photo, text, org, font, fontScale, color, thickness, cv2.LINE_AA)
            cv2.putText(photo, dis, (x1,y2-10), font, 0.65, (0,255,0), thickness)
            cv2.circle(photo, centroid[i-1], 4, (0, 255, 0), -1)
            cv2.putText(photo, f'Id: {i}', (int((x2+x1)/2)-10, int((y2+y1)/2)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            i += 1
        for i in range(len(centroid)):
            for j in range(i+1, len(centroid)):
                d = math.sqrt( ((centroid[j][1]-centroid[i][1])**2)+((centroid[j][0]-centroid[i][0])**2) )
                dP = "{:.2f}".format(17*400/d) + " cm"
                print("ID:",i+1,"- ID:",j+1,"=",dP)
                
                cv2.line(photo, (centroid[i][0], centroid[i][1]), (centroid[j][0], centroid[j][1]), (0, 0, 255), 2)
                cv2.putText(photo, dP, (int((centroid[i][0]+centroid[j][0])/2),int((centroid[i][1]+centroid[j][1])/2)-10), font, 0.65, (0,255,0), 2)
                if (17*400/d) < 40 :
                    person_in_contact += "Person "+str(i+1)+" and Person "+str(j+1)+" ; "
                    person_in_contact += " are not following social distancing "
                    print(person_in_contact)
                    print("!!  ALERT  !!")
        cv2.imshow('hi', photo)
        if cv2.waitKey(5) == 13:
            break
cv2.destroyAllWindows()

cap.release()

******************************************************
         Welcome to the Social Distancing App         
******************************************************


For this model initial calibration was done by taking width of object to be 17cm and the distance of 30cm away from camera so as to calculate the focal length. And then this focal length was used for calculating distance between the object and camera.
ID: 1 - ID: 2 = 25.96 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.74 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.55 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.35 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.33 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.21 cm
Person 1 and Person 2 ;  are not following social d

ID: 1 - ID: 2 = 25.37 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 32.03 cm
Person 1 and Person 2 ;  are not following social distancing Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 3 = 49.06 cm
ID: 1 - ID: 2 = 25.50 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 31.63 cm
Person 1 and Person 2 ;  are not following social distancing Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 3 = 48.91 cm
ID: 1 - ID: 2 = 25.69 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.63 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 50.02 cm
ID: 2 - ID: 3 = 31.63 cm
Person 1 and Person 2 ;  are not following social distancing Person 2 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 49.41 cm
ID: 1 - ID

ID: 1 - ID: 2 = 22.95 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 22.79 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 23.34 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 37.42 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 25.88 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 26.76 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 26.76 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 29.59 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 27.68 cm
Person 1 and Person 2 ;  are not following social distancing Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 4 = 33.38 cm
Perso

ID: 1 - ID: 2 = 43.19 cm
ID: 1 - ID: 2 = 39.76 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 35.97 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 22.45 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 43.54 cm
ID: 1 - ID: 3 = 22.34 cm
Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 4 = 37.77 cm
Person 1 and Person 3 ;  are not following social distancing Person 1 and Person 4 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 3 = 22.97 cm
Person 1 and Person 3 ;  are not following social distancing Person 1 and Person 4 ;  are not following social distancing Person 2 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 4 = 20.74 cm
Person 1 and Person 3 ;  are not following social distancing Person 1 and Person 4 ;  are not following social distancing Person 2 and P

ID: 1 - ID: 2 = 23.04 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 80.00 cm
ID: 1 - ID: 2 = 149.10 cm
ID: 1 - ID: 2 = 23.02 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 35.63 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 35.63 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 23.08 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 23.19 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 23.10 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 22.98 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 23.04 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 22.41 cm
Person 1 and Pe

ID: 1 - ID: 2 = 21.49 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.49 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.42 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.43 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.10 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 20.83 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.10 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.36 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 20.97 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 20.90 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  

ID: 1 - ID: 2 = 20.54 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.45 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 30.44 cm
Person 1 and Person 2 ;  are not following social distancing Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 3 = 57.29 cm
ID: 1 - ID: 2 = 21.86 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.59 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 3 = 31.75 cm
Person 1 and Person 2 ;  are not following social distancing Person 1 and Person 3 ;  are not following social distancing 
!!  ALERT  !!
ID: 2 - ID: 3 = 53.43 cm
ID: 1 - ID: 2 = 21.79 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 21.72 cm
Person 1 and Person 2 ;  are not following social distancing 
!!  ALERT  !!
ID: 1 - ID: 2 = 22.0